In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import train_test_split
from tqdm import tqdm
import gc
import time

from sklearn.preprocessing import PolynomialFeatures

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('loading files...')
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')

loading files...


In [3]:
train.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,target
0,cxcon_1,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2
1,cxcon_4,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
2,cxcon_7,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
3,cxcon_10,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
4,cxcon_13,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2


In [4]:
test.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23
0,cxcon_2,0,331,6085,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,13,13,255,255
1,cxcon_5,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255
2,cxcon_8,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255
3,cxcon_11,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,510,509,255,255
4,cxcon_14,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255


In [5]:
sample_submission.head()

,connection_id,target
0,cxcon_2,2
1,cxcon_5,0
2,cxcon_8,0
3,cxcon_11,1
4,cxcon_14,2


In [6]:
target = train['target']
test_ID = test['connection_id']
train.drop(['target','connection_id'] , axis=1 , inplace = True)
test.drop(['connection_id'] , axis=1 , inplace = True)
tr_ID = train.shape[0]

data = pd.concat([train, test] , axis = 0)

In [7]:
cont = data.columns[data.columns.str.startswith('cont')]
cat = data.columns[data.columns.str.startswith('cat')]

In [8]:
for col in tqdm(cat) :
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes

100%|██████████| 23/23 [00:00<00:00, 49.24it/s]


In [9]:
poly = PolynomialFeatures(degree = 2 ,interaction_only = True)
feat_new = poly.fit_transform(data[cont].values)
data = np.concatenate([data.values , feat_new] , axis = 1)
data = pd.DataFrame(data)

In [10]:
data['n0'] = (data==0).sum(axis=1)
data['n1'] = (data==1).sum(axis=1)
data['n2'] = (data==2).sum(axis=1)
data['n3'] = (data==3).sum(axis=1)
data['n4'] = (data==4).sum(axis=1)
data['n5'] = (data==5).sum(axis=1)
data['n6'] = (data==6).sum(axis=1)

In [11]:
train = data.iloc[:tr_ID]
test = data.iloc[tr_ID:]

In [13]:
# xgboost

t1 = time.time()

X = train
X = X.values
y = target.values

submission=test_ID.to_frame()
sub['target'] = 0 
nrounds = 3000
kfold = 5
#xgb
params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'multi:softprob', 'eval_metric': 'mlogloss', 'silent': True,'num_class':3}

skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100,  maximize=False, verbose_eval=10)
    
    l = xgb_model.predict(xgb.DMatrix(test.values), ntree_limit=xgb_model.best_ntree_limit+50)
    
    l[:,0] = l[:,0] - np.amin(l[:,0])/(np.amax(l[:,0]) - np.amin(l[:,0]))
    l[:,1] = l[:,1] - np.amin(l[:,1])/(np.amax(l[:,1]) - np.amin(l[:,1]))
    l[:,2] = l[:,2] - np.amin(l[:,2])/(np.amax(l[:,2]) - np.amin(l[:,2]))
    sub += l/(kfold)
    
t2 = time.time()
print "Time taken {} hr".format((t2-t1)/(60*60))

 xgb kfold: 1  of  5 : 
[0]	train-mlogloss:1.08334	valid-mlogloss:1.08333
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[10]	train-mlogloss:0.955528	valid-mlogloss:0.955422
[20]	train-mlogloss:0.861155	valid-mlogloss:0.860952
[30]	train-mlogloss:0.789512	valid-mlogloss:0.789266
[40]	train-mlogloss:0.734322	valid-mlogloss:0.734012
[50]	train-mlogloss:0.690613	valid-mlogloss:0.690328
[60]	train-mlogloss:0.655525	valid-mlogloss:0.655308
[70]	train-mlogloss:0.627184	valid-mlogloss:0.627063
[80]	train-mlogloss:0.603969	valid-mlogloss:0.60396
[90]	train-mlogloss:0.584965	valid-mlogloss:0.585059
[100]	train-mlogloss:0.569236	valid-mlogloss:0.56943
[110]	train-mlogloss:0.556192	valid-mlogloss:0.5565
[120]	train-mlogloss:0.545244	valid-mlogloss:0.545654
[130]	train-mlogloss:0.536023	valid-mlogloss:0.536508
[140]	train-mlogloss:0.528185	valid-mlogloss:0.528732
[150]	train-mlogloss:0.521461

NameError: name 'sub' is not defined

In [ ]:
s = np.zeros([test.shape[0]])
for i in tqdm(range(test.shape[0])) :
    s[i] = sub[i].argmax()

In [ ]:
submission=test_ID.to_frame()
submission['target'] = s
submission.to_csv('xgb_update1.csv', index=False, float_format='%.5f')